In [ ]:
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd 
import os 
import plotly.io as pio
pio.kaleido.scope.mathjax = None 
from math import log

# <b>Functions for visualizing benchmarking results</b>

## Speedup Visualization

In [ ]:
def find_color(line_color_map, imp):
    for i, c in line_color_map:
        if i == imp:
            return c 

def vis_bench_speedup(primary: str, secondary: str, date: str, time: str, save_pdf: bool = False):
    df = pd.read_csv("csv_results/" + date + "/" + primary + "_" + secondary + "_" + date + "_" + time + ".csv")
    df.sort_values('set_size', inplace=True)

    exponentials = []
    i = 1
    for _ in df.set_size.unique():
        exponentials.append("2^" + str(i))
        i += 1

    fig = make_subplots(rows=3, 
                        cols=1, shared_yaxes=False, shared_xaxes=True,
                        x_title="Number of VLMCs",
                        y_title="Speedup",
                        subplot_titles=("Small", "Medium", "Large"), 
                        horizontal_spacing= 0.02, vertical_spacing= 0.05) # specs=[[{}, {}],[{"colspan": 2}, None]])
    
    df_pst = df[df.implementation=="PstClassifierSeqan"]
    df = df[df.implementation!="PstClassifierSeqan"]

    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'pentagon', 'star', 'diamond-wide']
    line_dashes = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    implementations = ["PstClassifierSeqan", "sorted-vector", "sorted-search", "hashmap", "veb", "ey", "alt-btree" ,"sorted-search-ey"]
    line_color_map = list(zip(implementations, px.colors.qualitative.Plotly))
    iterate_on = list(zip(list(df.implementation.unique()), marker_symbols, line_dashes))
    for imp, marker_symbol, line_dash in iterate_on:
        df_imp = df[df.implementation==imp]
        line_color = find_color(line_color_map, imp)
        fig.add_trace(go.Scatter(x=exponentials, y=list(df_pst[df_pst.vlmc_size=='small']['elapsed_time']) / df_imp[df_imp.vlmc_size=='small']['elapsed_time'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash), 1, 1)
        fig.add_trace(go.Scatter(x=exponentials, y=list(df_pst[df_pst.vlmc_size=='medium']['elapsed_time']) / df_imp[df_imp.vlmc_size=='medium']['elapsed_time'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, showlegend = False), 2, 1)
        fig.add_trace(go.Scatter(x=exponentials, y=list(df_pst[df_pst.vlmc_size=='large']['elapsed_time']) / df_imp[df_imp.vlmc_size=='large']['elapsed_time'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, showlegend = False), 3, 1)
    
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        font_size=24,
        y=1.08),
        margin=dict(l=100, r=60, t=60, b=80),
        height=900,
        width=900,
        plot_bgcolor="white")
    fig.update_layout(xaxis_range=[0, i - 1.5])

    fig.update_xaxes(gridcolor='LightGrey', tickfont_size=20, showline=True, linewidth=1, linecolor='LightGrey')
    fig.update_yaxes(gridcolor='LightGrey', tickfont_size=20, showline=True, linewidth=1, linecolor='LightGrey')

    for i in range(0, len(fig.layout.annotations)):
        fig.layout.annotations[i]["font"] = {'size': 24}
    
    if save_pdf:
        fig.write_image("images/" + primary + "_to_" + secondary + "_elapsed_time.pdf") 
    fig.show()

## Cache-misses Visualization

In [ ]:
def find_color(line_color_map, imp):
    for i, c in line_color_map:
        if i == imp:
            return c 

def vis_bench_cache_misses(primary: str, secondary: str, date: str, time: str, save_pdf: bool = False):
    df = pd.read_csv("csv_results/" + date + "/" + primary + "_" + secondary + "_" + date + "_" + time + ".csv")
    df.sort_values('set_size', inplace=True)

    exponentials = []
    i = 1
    for _ in df.set_size.unique():
        exponentials.append("2^" + str(i))
        i += 1

    fig = make_subplots(rows=3, 
                        cols=1, shared_yaxes=False, shared_xaxes=True,
                        x_title="Number of VLMCs",
                        y_title="Cache misses (%)",
                        subplot_titles=("Small", "Medium", "Large"), 
                        horizontal_spacing= 0.02, vertical_spacing= 0.05) # specs=[[{}, {}],[{"colspan": 2}, None]])
    
    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'pentagon', 'star']#, 'diamond-wide']
    line_dashes = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot', 'dot']
    implementations = ["PstClassifierSeqan", "sorted-vector", "sorted-search", "hashmap", "veb", "ey", "alt-btree"]
    line_color_map = list(zip(implementations, px.colors.qualitative.Plotly))
    iterate_on = list(zip(list(df.implementation.unique()), marker_symbols, line_dashes))
    for imp, marker_symbol, line_dash in iterate_on:
        df_imp = df[df.implementation==imp]
        line_color = find_color(line_color_map, imp)
        fig.add_trace(go.Scatter(x=exponentials, y=df_imp[df_imp.vlmc_size=='small']['cache_misses'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash), 1, 1)
        fig.add_trace(go.Scatter(x=exponentials, y=df_imp[df_imp.vlmc_size=='medium']['cache_misses'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, showlegend = False), 2, 1)
        fig.add_trace(go.Scatter(x=exponentials, y=df_imp[df_imp.vlmc_size=='large']['cache_misses'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, showlegend = False), 3, 1)
    
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        font_size=24,
        y=1.08),
        margin=dict(l=100, r=60, t=60, b=80),
        height=900,
        width=900,
        plot_bgcolor="white")
    fig.update_layout(xaxis_range=[0, i - 1.5])

    fig.update_xaxes(gridcolor='LightGrey', tickfont_size=20, showline=True, linewidth=1, linecolor='LightGrey')
    fig.update_yaxes(gridcolor='LightGrey', tickfont_size=20, showline=True, linewidth=1, linecolor='LightGrey')

    for i in range(0, len(fig.layout.annotations)):
        fig.layout.annotations[i]["font"] = {'size': 24}
    
    if save_pdf:
        fig.write_image("images/" + primary + "_to_" + secondary + "_cache_misses.pdf") 
    fig.show()

# Single Run Visualization

In [ ]:
def find_color(line_color_map, imp):
    for i, c in line_color_map:
        if i == imp:
            return c 

def vis_bench_speedup_single_run(date: str, time: str, save_pdf: bool = False):
    datasets = [("human", "human"), ("human", "turkey") , ("human", "corn"), ("turkey", "turkey"), ("turkey", "corn"), ("corn", "corn")] # , ("ecoli", "ecoli")]

    df_list = []
    for primary, secondary in datasets:
        df_tmp = pd.read_csv("csv_results/" + date + "/" + primary + "_" + secondary + "_" + date + "_" + time + ".csv")
        for s in ["small", "medium", "large"]:
            df_pst_time = df_tmp[(df_tmp.vlmc_size==s) & (df_tmp.implementation=="PstClassifierSeqan")]['elapsed_time'].iloc[0]
            df_tmp.loc[df_tmp.vlmc_size==s, 'speedup'] = df_pst_time / df_tmp[df_tmp.vlmc_size==s].elapsed_time 
        df_tmp['datasets'] = primary + " to " + secondary 
        df_list.append(df_tmp)
    
    df_tmp = pd.read_csv("csv_results/05_04/ecoli_ecoli_05_04_10_09.csv")
    for s in ["small", "medium", "large"]:
        df_pst_time = df_tmp[(df_tmp.vlmc_size==s) & (df_tmp.implementation=="PstClassifierSeqan")]['elapsed_time'].iloc[0]
        df_tmp.loc[df_tmp.vlmc_size==s, 'speedup'] = df_pst_time / df_tmp[df_tmp.vlmc_size==s].elapsed_time 
    df_tmp['datasets'] = "ecoli" + " to " + "ecoli" 
    df_list.append(df_tmp)
    df = pd.concat(df_list)

    fig = make_subplots(rows=3, 
                        cols=1, shared_yaxes=False, shared_xaxes=True,
                        x_title="",
                        y_title="Speedup",
                        subplot_titles=("Small", "Medium", "Large"), 
                        horizontal_spacing= 0.02, vertical_spacing= 0.05) # specs=[[{}, {}],[{"colspan": 2}, None]])

    marker_size = 12

    df = df[df.implementation!="PstClassifierSeqan"]
    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'pentagon', 'star', 'diamond-wide']
    line_dashes = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    implementations = ["PstClassifierSeqan", "sorted-vector", "sorted-search", "hashmap", "veb", "ey", "alt-btree" ,"sorted-search-ey"]
    line_color_map = list(zip(implementations, px.colors.qualitative.Plotly))
    iterate_on = list(zip(list(df.implementation.unique()), marker_symbols, line_dashes))

    for imp, marker_symbol, line_dash in iterate_on:
        df_imp = df[df.implementation==imp]
        line_color = find_color(line_color_map, imp)
        fig.add_trace(go.Scatter(x=df_imp[df_imp.vlmc_size=='small']['datasets'], y=df_imp[df_imp.vlmc_size=='small']['speedup'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, mode='markers', marker_size=marker_size), 1, 1)
        fig.add_trace(go.Scatter(x=df_imp[df_imp.vlmc_size=='medium']['datasets'], y=df_imp[df_imp.vlmc_size=='medium']['speedup'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, showlegend = False, mode='markers', marker_size=marker_size), 2, 1)
        fig.add_trace(go.Scatter(x=df_imp[df_imp.vlmc_size=='large']['datasets'], y=df_imp[df_imp.vlmc_size=='large']['speedup'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, showlegend = False, mode='markers', marker_size=marker_size), 3, 1)

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        font_size=24,
        y=1.08),
        margin=dict(l=100, r=60, t=60, b=80),
        height=900,
        width=900,
        plot_bgcolor="white")
    fig.update_xaxes(gridcolor='LightGrey', tickfont_size=20, showline=True, linewidth=1, linecolor='LightGrey')
    fig.update_yaxes(gridcolor='LightGrey', tickfont_size=20, showline=True, linewidth=1, linecolor='LightGrey')
    for i in range(0, len(fig.layout.annotations)):
        fig.layout.annotations[i]["font"] = {'size': 24}

    if save_pdf:
        fig.write_image("images/Single_run_" + date + "_" + time + "_elapsed_time.pdf") 
    fig.show()

In [ ]:
def find_color(line_color_map, imp):
    for i, c in line_color_map:
        if i == imp:
            return c 

def vis_bench_cache_misses_single_run(date: str, time: str, save_pdf: bool = False):
    datasets = [("human", "human"), ("human", "turkey") , ("human", "corn"), ("turkey", "turkey"), ("turkey", "corn"), ("corn", "corn")] # , ("ecoli", "ecoli")]

    df_list = []
    for primary, secondary in datasets:
        df_tmp = pd.read_csv("csv_results/" + date + "/" + primary + "_" + secondary + "_" + date + "_" + time + ".csv")
        df_tmp['misses_per_instruction'] = df_tmp.cache_misses_count / df_tmp.instructions_count
        df_tmp['datasets'] = primary + " to " + secondary 
        df_list.append(df_tmp)

    df = pd.concat(df_list)

    fig = make_subplots(rows=3, 
                        cols=1, shared_yaxes=False, shared_xaxes=True,
                        x_title="",
                        y_title="Speedup",
                        subplot_titles=("Small", "Medium", "Large"), 
                        horizontal_spacing= 0.02, vertical_spacing= 0.05) # specs=[[{}, {}],[{"colspan": 2}, None]])

    marker_size = 12

    marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'pentagon', 'star', 'diamond-wide']
    line_dashes = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    implementations = ["PstClassifierSeqan", "sorted-vector", "sorted-search", "hashmap", "veb", "ey", "alt-btree" ,"sorted-search-ey"]
    line_color_map = list(zip(implementations, px.colors.qualitative.Plotly))
    iterate_on = list(zip(list(df.implementation.unique()), marker_symbols, line_dashes))

    for imp, marker_symbol, line_dash in iterate_on:
        df_imp = df[df.implementation==imp]
        line_color = find_color(line_color_map, imp)
        fig.add_trace(go.Scatter(x=df_imp[df_imp.vlmc_size=='small']['datasets'], y=df_imp[df_imp.vlmc_size=='small']['misses_per_instruction'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, mode='markers', marker_size=marker_size), 1, 1)
        fig.add_trace(go.Scatter(x=df_imp[df_imp.vlmc_size=='medium']['datasets'], y=df_imp[df_imp.vlmc_size=='medium']['misses_per_instruction'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, showlegend = False, mode='markers', marker_size=marker_size), 2, 1)
        fig.add_trace(go.Scatter(x=df_imp[df_imp.vlmc_size=='large']['datasets'], y=df_imp[df_imp.vlmc_size=='large']['misses_per_instruction'], line_color=line_color, name=imp, marker_symbol=marker_symbol, line_dash=line_dash, showlegend = False, mode='markers', marker_size=marker_size), 3, 1)

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        font_size=24,
        y=1.08),
        margin=dict(l=100, r=60, t=60, b=80),
        height=900,
        width=900,
        plot_bgcolor="white")
    fig.update_xaxes(gridcolor='LightGrey', tickfont_size=20, showline=True, linewidth=1, linecolor='LightGrey')
    fig.update_yaxes(gridcolor='LightGrey', tickfont_size=20, showline=True, linewidth=1, linecolor='LightGrey')
    for i in range(0, len(fig.layout.annotations)):
        fig.layout.annotations[i]["font"] = {'size': 24}

    if save_pdf:
        fig.write_image("images/Single_run_" + date + "_" + time + "_cache_misses.pdf") 
    fig.show()

# <b>PLOTS</b>

In [ ]:
vis_bench_speedup_single_run("05_05", "00_29", True)
vis_bench_cache_misses_single_run("05_05", "00_29", True)

In [ ]:
print("Speedup Corn to Corn")
vis_bench_speedup("corn", "corn", "05_02", "14_31")
print("Cache-misses Corn to Corn")
vis_bench_cache_misses("corn", "corn", "05_02", "14_31")

In [ ]:
print("Speedup Ecoli to Ecoli")
vis_bench_speedup("ecoli", "ecoli", "05_02", "21_55")
print("Cache-misses ecoli to Ecoli")
vis_bench_cache_misses("ecoli", "ecoli", "05_02", "21_55")

In [ ]:
print("Speedup Human to Corn")
vis_bench_speedup("human", "corn", "05_02", "14_31")
print("Cache-misses Human to Corn")
vis_bench_cache_misses("human", "corn", "05_02", "14_31")

In [ ]:
print("Speedup Human to Human")
vis_bench_speedup("human", "human", "05_02", "14_31")
print("Cache-misses Human to Human")
vis_bench_cache_misses("human", "human", "05_02", "14_31")

In [ ]:
print("Speedup Human to Turkey")
vis_bench_speedup("human", "turkey", "05_02", "14_31")
print("Cache-misses Human to Turkey")
vis_bench_cache_misses("human", "turkey", "05_02", "14_31")

In [ ]:
print("Speedup Turkey to Corn")
vis_bench_speedup("turkey", "corn", "05_02", "14_31")
print("Cache-misses Turkey to Corn")
vis_bench_cache_misses("turkey", "corn", "05_02", "14_31")

In [ ]:
print("Speedup Turkey to Turkey")
vis_bench_speedup("turkey", "turkey", "05_02", "14_31")
print("Cache-misses Turkey to Turkey")
vis_bench_cache_misses("turkey", "turkey", "05_02", "14_31")